In [32]:
# fetch input data: Hungarian, Gothic, West Old Turkic; German Word vectors

from shutil import copy

copy("../gerstnerhungarian/cldf/forms.csv", "hun_forms.csv")  # recipient lg df
copy("../streitberggothic/cldf/forms.csv", "got_forms.csv")  # donor lg df
copy("../ronataswestoldturkic/cldf/forms.csv", "wot_forms.csv")  # etymological data
copy("../GothicHungarian_backup/german.model", "german.model")  # word vectors

'german.model'

In [ ]:
# download German word-vectors

import wget

wget.download("https://cloud.devmount.de/d2bc5672c523b086/german.model")

In [33]:
# import Etymology class from helpers module

from loanpy.helpers import Etym

# turn etymological data into a Python object

etym_obj = Etym(forms_csv="../ronataswestoldturkic/cldf/forms.csv",
                source_language="WOT",
                target_language="EAH")

In [35]:
# create dictionary of heuristic sound substitutions

etym_obj.get_scdictbase(write_to="scWOT2EAH.txt")  # 1.7MB! 

100%|█████████████████████████████████████| 6489/6489 [00:02<00:00, 2530.36it/s]


{'˩': ['r',
  'ø',
  'u',
  'ɯ',
  'aː',
  'uː',
  'ɛ',
  'eː',
  'y',
  'yː',
  'e',
  'i',
  'a',
  'iː',
  'øː',
  'o',
  'ɒ',
  'h',
  'ɟ',
  'm',
  'ɡ',
  'w',
  'ɣ',
  'b',
  'mː',
  'p',
  'ɲ',
  'j',
  'k',
  'ŋ',
  'β',
  'l',
  'z',
  'n',
  'd',
  'd͡ʒʲ',
  'ʃ',
  't',
  's',
  'v',
  'lː',
  'tː',
  't͡ʃ'],
 '˨': ['r',
  'ø',
  'u',
  'ɯ',
  'aː',
  'uː',
  'ɛ',
  'eː',
  'y',
  'yː',
  'e',
  'i',
  'a',
  'iː',
  'øː',
  'o',
  'ɒ',
  'h',
  'ɟ',
  'm',
  'ɡ',
  'w',
  'ɣ',
  'b',
  'mː',
  'p',
  'ɲ',
  'j',
  'k',
  'ŋ',
  'β',
  'l',
  'z',
  'n',
  'd',
  'd͡ʒʲ',
  'ʃ',
  't',
  's',
  'v',
  'lː',
  'tː',
  't͡ʃ'],
 '˧': ['r',
  'ø',
  'u',
  'ɯ',
  'aː',
  'uː',
  'ɛ',
  'eː',
  'y',
  'yː',
  'e',
  'i',
  'a',
  'iː',
  'øː',
  'o',
  'ɒ',
  'h',
  'ɟ',
  'm',
  'ɡ',
  'w',
  'ɣ',
  'b',
  'mː',
  'p',
  'ɲ',
  'j',
  'k',
  'ŋ',
  'β',
  'l',
  'z',
  'n',
  'd',
  'd͡ʒʲ',
  'ʃ',
  't',
  's',
  'v',
  'lː',
  'tː',
  't͡ʃ'],
 '˦': ['r',
  'ø',
  'u',
  'ɯ',
  'a

In [36]:
# import Quantify class from quantify-sound-correspondences module

from loanpy.qfysc import Qfy

# turn etymological data into a Python object

qfy_obj = Qfy(forms_csv="wot_forms.csv",
              source_language="WOT",
              target_language="EAH",
              scdictbase="scWOT2EAH.txt",
              mode="adapt")

# extract sound substitutions from file, add to heuristics, overwrite sound-correspondence-file

qfy_obj.get_sound_corresp("scWOT2EAH.txt")

[{'C': ['ɡ',
   'n',
   'm',
   't',
   's',
   'j',
   'ɣ',
   'p',
   'w',
   'h',
   'l',
   'r',
   'ŋ',
   'lː',
   'ɲ',
   'mː',
   'β',
   'z',
   'v',
   'ʃ',
   'k',
   'ɟ',
   'b',
   'd',
   'tː',
   't͡ʃ',
   'd͡ʒʲ'],
  'V': ['a',
   'i',
   'ɛ',
   'u',
   'ɯ',
   'e',
   'o',
   'ø',
   'eː',
   'ɒ',
   'aː',
   'iː',
   'øː',
   'uː',
   'y',
   'yː'],
  'a': ['a',
   'aː',
   'o',
   'ɒ',
   '',
   'i',
   'u',
   'e',
   'ɛ',
   'ø',
   'eː',
   'iː',
   'øː',
   'ɯ',
   'y',
   'yː',
   'uː',
   'j',
   'w',
   'h',
   'l',
   'r',
   'lː',
   'm',
   'ɲ',
   'n',
   'mː',
   'ŋ',
   'β',
   'z',
   'v',
   'ʃ',
   's',
   'ɣ',
   'ɟ',
   'b',
   'd',
   'p',
   't',
   'ɡ',
   'k',
   'tː',
   't͡ʃ',
   'd͡ʒʲ'],
  'aː': ['a',
   'aː',
   'eː',
   'e',
   'iː',
   'øː',
   'ɛ',
   'ø',
   'i',
   'ɒ',
   'yː',
   'o',
   'ɯ',
   'uː',
   'y',
   'u',
   'j',
   'w',
   'h',
   'lː',
   'l',
   'r',
   'mː',
   'm',
   'ɲ',
   'n',
   'ŋ',
   'β',
   'z',
   'v',
   'ʃ

In [37]:
# import Quantify class from quantify-sound-correspondences module

from loanpy.qfysc import Qfy

# turn etymological data into a Python object

qfy_obj = Qfy(forms_csv="wot_forms.csv",
              source_language="H",
              target_language="EAH",
              mode="reconstruct")

# extract historical sound changes

qfy_obj.get_sound_corresp("scH2EAH.txt")

[{'#-': ['-', 'v'],
  '#aː': ['a', 'aː', 'o'],
  '#b': ['b'],
  '#c': ['t'],
  '#d': ['d', 'd͡ʒʲ', 't'],
  '#e': ['e'],
  '#eː': ['ɛ'],
  '#h': ['h', 'ks', '-'],
  '#i': ['i', 'y', 'ɛ'],
  '#iː': ['i'],
  '#k': ['k'],
  '#l': ['v'],
  '#m': ['m'],
  '#o': ['u', 'o', 'a'],
  '#oː': ['a'],
  '#s': ['s'],
  '#t': ['t'],
  '#t͡s': ['s', 't͡ʃ'],
  '#t͡ʃ': ['t͡ʃ'],
  '#u': ['u'],
  '#uː': ['u'],
  '#v': ['v'],
  '#y': ['i', 'ɛ', 'y', 'ø'],
  '#z': ['s'],
  '#ø': ['y', 'ɛ', 'ø', 'i'],
  '#øː': ['ɛ'],
  '#ɒ': ['a', 'o'],
  '#ɛ': ['ɛ', 'i'],
  '#ɟ': ['d͡ʒʲ', 'ɟ'],
  '#ɡ': ['ɡ', 'k'],
  '#ɲ': ['n'],
  '#ʃ': ['ʃ', 't͡ʃ', 's'],
  '-#': ['-',
   'ɣ',
   'i',
   'iɣ',
   'y',
   'ɛ',
   'an',
   'u',
   'ur',
   'ɣu',
   'at',
   'in',
   'ɛk',
   'v',
   'øk',
   'ɛn',
   'yr',
   'ɣaɣ',
   'ɣy',
   'j',
   'ɒ',
   'uk',
   'at͡ʃi',
   'ruɣ',
   'l',
   'ut',
   'ol',
   'yt͡ʃi',
   'iri',
   'yz',
   'il',
   'ɛɡ',
   'kɛ',
   'rɛɣ',
   'n',
   'iz',
   'ørøk',
   'yɣ',
   'ɡɛliɣ',
   'ul',
   'ir

In [38]:
# import Adrc class from adapt-reconstruct module

from loanpy.adrc import Adrc

# read sound correspondence data, create Python object from it

ad_obj = Adrc(scdictlist="scWOT2EAH.txt",
              forms_csv="wot_forms.csv",
              source_language="WOT",
              target_language="EAH",
              mode="adapt")

In [70]:
# use tool to make predictions about loanword adaptation

# /ɸuɣl̥s/ "bird", /'laɪpt͡sʃ'/, /bɔ̃ʒuʀ/ "good day", 

ad_obj.adapt(ipastr="laɪpt͡sʃ",
             howmany=10,
             max_repaired_phonotactics=2,
             max_paths2repaired_phonotactics=1,
             sort_by_nse=False,
             show_workflow=True)

'laɡipatʃ, laɡipitʃ, laɡipɛtʃ, laɡiputʃ, laɡipɯtʃ, lanipatʃ, lanipitʃ, lanipɛtʃ, laniputʃ, lanipɯtʃ'

In [58]:
# white-box: inspect what happened

ad_obj.workflow

OrderedDict([('tokenised', "['b', 'ɔ̃', 'ʒ', 'u', 'ʀ']"),
             ('donor_phonotactics', 'CVCVC'),
             ('predicted_phonotactics', "['CVCVC', 'CVC']"),
             ('adapted_phonotactics',
              "[['b', 'ɔ̃', 'ʒ', 'u', 'ʀ'], ['ʒ', 'u', 'ʀ'], ['b', 'u', 'ʀ']]"),
             ('before_combinatorics',
              "[[['b', 'p', 'β', 'd'], ['o', 'ø'], ['ʃ'], ['u', 'o', ''], ['h']], [['ʃ', 'z'], ['u', 'o', '', 'i'], ['h', 'r']], [['b', 'p', 'β', 'd'], ['u', 'o', '', 'i'], ['h']]]")])

In [29]:
# orthography to ipa. hun-Latn, deu-Latn, etc.
from epitran import Epitran
epi = Epitran("deu-Latn")
epi.transliterate("Leiptz sch")  # hɪrɪ aːɟuː tɛlɛviːzioː baːɡɛr

'laɪpt͡s ʃ'

In [15]:
# read etymological data for reconstructions

rc_obj = Adrc(scdictlist="scH2EAH.txt",
              forms_csv="wot_forms.csv",
              source_language="H",
              target_language="EAH",
              mode="reconstruct")

In [20]:
# predict reconstructions

rc_obj.reconstruct(ipastring="baːɡɛr",
                   howmany=100,
                   vowelharmony_filter=True,
                   phonotactics_filter=True,
                   clusterised=False)

'^baɡɛr$|^baɡɛv$|^baɡɛɣ$|^baɡɛt$|^baɡir$|^baɡiv$|^baɡiɣ$|^baɡit$|^baɡør$|^baɡøv$|^baɡøɣ$|^baɡøt$|^baɡyr$|^baɡyv$|^baɡyɣ$|^baɡyt$|^baɡar$|^baɡav$|^baɡaɣ$|^baɡat$|^banɛr$|^banɛv$|^banɛɣ$|^banɛt$|^banir$|^baniv$|^baniɣ$|^banit$|^banør$|^banøv$|^banøɣ$|^banøt$|^banyr$|^banyv$|^banyɣ$|^banyt$|^banar$|^banav$|^banaɣ$|^banat$|^baɡlɛr$|^baɡlɛv$|^baɡlɛɣ$|^baɡlɛt$|^baɡlir$|^baɡliv$|^baɡliɣ$|^baɡlit$|^baɡlør$|^baɡløv$|^baɡløɣ$|^baɡløt$|^baɡlyr$|^baɡlyv$|^baɡlyɣ$|^baɡlyt$|^baɡlar$|^baɡlav$|^baɡlaɣ$|^baɡlat$|^baːɡɛr$|^baːɡɛv$|^baːɡɛɣ$|^baːɡɛt$|^baːɡir$|^baːɡiv$|^baːɡiɣ$|^baːɡit$|^baːɡør$|^baːɡøv$|^baːɡøɣ$|^baːɡøt$|^baːɡyr$|^baːɡyv$|^baːɡyɣ$|^baːɡyt$|^baːɡar$|^baːɡav$|^baːɡaɣ$|^baːɡat$|^baːnɛr$|^baːnɛv$|^baːnɛɣ$|^baːnɛt$|^baːnir$|^baːniv$|^baːniɣ$|^baːnit$|^baːnør$|^baːnøv$|^baːnøɣ$|^baːnøt$|^baːnyr$|^baːnyv$|^baːnyɣ$|^baːnyt$|^baːnar$|^baːnav$|^baːnaɣ$|^baːnat$'

In [ ]:
# evaluate sanity of reconstruction model
# and optimise parameters

from loanpy.sanity import eval_all, cache
from loanpy.sanity import ArgumentsAlreadyTested

eval_all = cache(eval_all)

for cl in [True, False]:
    for ph_f in [True, False]:
        for vh in [True, False]:
            try:
                eval_all(forms_csv="ronataswestoldturkic/cldf/forms.csv",
                         source_language="H",
                         target_language="EAH",
                         mode="reconstruct",
                         write_to="GothicHungarian/eval_rc.csv",
                         path2cache="GothicHungarian/cache_rc.csv",
                         clusters=cl,
                         vowelharmony=vh,
                         phonotactics_filter=ph_f)
            except ArgumentsAlreadyTested:
                print("arguments already tested")

In [ ]:
# evaluate sanity of adaptation model
# and optimise parameters

from loanpy.sanity import eval_all, cache

eval_all = cache(eval_all)

for cl in [True, False]:
    for ph_f in [True, False]:
        for vh in [True, False]:
            eval_all(forms_csv="ronataswestoldturkic/cldf/forms.csv",
                     source_language="WOT",
                     target_language="EAH",
                     mode="adapt",
                     write_to="GothicHungarian/eval_ad.csv",
                     path2cache="GothicHungarian/cache_ad.csv",
                     clusters=cl,
                     vowelharmony=vh,
                     phonotactics_filter=ph_f)

In [ ]:
# evaluate sanity of adaptation model
# and optimise parameters

from loanpy.sanity import eval_all, cache

eval_all = cache(eval_all)

for mrp in [2, 10, 20]:
    for mp2rp in [2, 10, 20]:
        eval_all(forms_csv="ronataswestoldturkic/cldf/forms.csv",
                 source_language="WOT",
                 target_language="EAH",
                 mode="adapt",
                 write_to="GothicHungarian/eval_ad.csv",
                 path2cache="GothicHungarian/cache_ad2.csv",
                 clusters=False,
                 vowelharmony=False,
                 phonotactics_filter=False,
                 max_repaired_phonotactics=mrp,
                 max_paths2repaired_phonotactics=mp2rp)

In [ ]:
# evaluate sanity
# and optimise parameters

from loanpy.sanity import eval_all, cache

eval_all = cache(eval_all)

for mrp in [3, 5, 8]:
    for mp2rp in [3, 5, 8]:
        eval_all(forms_csv="ronataswestoldturkic/cldf/forms.csv",
                 source_language="WOT",
                 target_language="EAH",
                 mode="adapt",
                 write_to="GothicHungarian/eval_ad.csv",
                 path2cache="GothicHungarian/cache_ad2.csv",
                 clusters=False,
                 vowelharmony=False,
                 phonotactics_filter=False,
                 max_repaired_phonotactics=mrp,
                 max_paths2repaired_phonotactics=mp2rp)

In [ ]:
# create adaptations

from loanpy.adrc import Adrc
import pandas as pd

# read sound correspondence data, create Python object from it
ad_obj = Adrc(scdictlist="scWOT2EAH.txt",
              forms_csv="wot_forms.csv",
              source_language="WOT",
              target_language="EAH",
              mode="adapt")

# read in forms.csv with pandas
dfforms = pd.read_csv("got_forms.csv")

# add new column of pseudo-adaptations from Gothic into Hungarian with loanpy
output = [(ad_obj.adapt(word.replace(" ", ""),
                        howmany=1000,
                        max_repaired_phonotactics=2,
                        max_paths2repaired_phonotactics=2,
                        show_workflow=True), ad_obj.workflow)
          for word in dfforms["Segments"]]

dfforms["ad"] = [i[0] for i in output]
dfforms["ad_workflow"] = [i[1] for i in output]

# write to file
dfforms.to_csv("ad_got_forms.csv", encoding="utf-8", index=False)

In [ ]:
# create reconstructions

from loanpy.adrc import Adrc
import pandas as pd

# read etymological data for reconstructions
rc_obj = Adrc(scdictlist="scH2EAH.txt",
              forms_csv="hun_forms.csv",
              source_language="H",
              target_language="EAH",
              mode="reconstruct")

# read in forms.csv with pandas
dfforms = pd.read_csv("hun_forms.csv")

#drop years after 1600

# add new column of pseudo-reconstructions from Hungarian into Early Ancient Hungarian with loanpy
dfforms["rc"] = [rc_obj.reconstruct(word.replace(" ", ""),
                                    howmany=1000,
                                    vowelharmony_filter=False,
                                    phonotactics_filter=False,
                                    clusterised=True)
                       for word in dfforms["Segments"]]

# write to file
dfforms.to_csv("rc_hun_forms.csv", encoding="utf-8", index=False)

In [ ]:
from gensim.models import KeyedVectors

from loanpy.helpers import plug_in_model
from loanpy.loanfinder import Search

#create instance of loanpy.loanfinder.Search
search_obj = Search(
path2donordf="ad_got_forms.csv", path2recipdf="rc_hun_forms.csv",
semsim=0, scdictlist_ad="scWOT2EAH.txt", scdictlist_rc="scH2EAH.txt")

# plug in German vectors manually, since default is English
plug_in_model(KeyedVectors.load_word2vec_format("german.model", binary=True))

# search for loans
search_obj.loans(write_to="loans.csv")